In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import os

from astropy.io import fits
from astropy.table import Table
import stuff

## Dump configuration

In [2]:
print(open(os.path.expanduser('~/Work/Data/sim08/test_py.config')).read())

output-properties=WorldCentroid,PixelCentroid,SourceIDs,AutoPhotometry
#,FlexibleModelFitting
#,AutoPhotometry
detection-image=/home/aalvarez/Work/Data/sim08/coadd.fits
weight-image=/home/aalvarez/Work/Data/sim08/coadd.weight.fits
weight-type=weight
detection-image-saturation=59000
detection-image-gain=1
detect-minarea=10
partition-min-area=10
threads-nb=4
cellsize-value=32
partition-multithreshold=true
grouping-algorithm=SPLIT
checkimage-residual=/home/aalvarez/Work/Data/sim08/mci_residual.fits
checkimage-modelfitting=/home/aalvarez/Work/Data/sim08/mci_modelfitting.fits
checkimage-background=/home/aalvarez/Work/Data/sim08/mci_background.fits
checkimage-variance=/home/aalvarez/Work/Data/sim08/mci_variance.fits
checkimage-segmentation=/home/aalvarez/Work/Data/sim08/mci_segmentation.fits
checkimage-partition=/home/aalvarez/Work/Data/sim08/mci_partition.fits
checkimage-filtered=/home/aalvarez/Work/Data/sim08/mci_filtered.fits
checkimage-aperture=/home/aalvarez/Work/Data/sim08/mci_aperture

In [3]:
print(open(os.path.expanduser('~/Work/Projects/sextractorxx/tests/test_config.py')).read())

from os.path import expanduser
import sys

sys.path.append(expanduser('~/Work/Projects/sextractorxx/SEImplementation/python'))
sys.path.append(expanduser('~/Work/Projects/sextractorxx/build.x86_64-fc28-gcc82-dbg/lib'))

from sextractorxx.config import *
from glob import glob

images = sorted(glob(expanduser('~/Work/Data/sim08/sim08_[r|i]_0[1|2].fits')))
psfs = sorted(glob(expanduser('~/Work/Data/sim08/sim08_[r|i]_0[1|2].psf')))

top = load_fits_images(images, psfs)
top.split(ByKeyword('FILTER'))

measurement_group = MeasurementGroup(top)

alpha, delta = get_pos_parameters()
angle = FreeParameter(lambda o: o.get_angle(), Range((-2 * 3.14159, 2 * 3.14159), RangeType.LINEAR))
ratio = FreeParameter(1, Range((0, 10), RangeType.LINEAR))
rad = FreeParameter(lambda o: o.get_radius_world(), Range(lambda v,o: (.01*v, 100*v), RangeType.EXPONENTIAL))

add_output_column('alpha', alpha)
add_output_column('delta', delta)

for band, group in top:
    all_ap = []
    for img in group:
        if band =

## Output catalog

In [4]:
catalog = Table.read(os.path.expanduser('~/Work/Data/sim08/py_output.fits'))
catalog.colnames

['aperture_i_flux',
 'aperture_r_flux',
 'aperture_i_flux_err',
 'aperture_r_flux_err',
 'aperture_i_mag',
 'aperture_r_mag',
 'aperture_i_mag_err',
 'aperture_r_mag_err',
 'aperture_i_flags',
 'aperture_r_flags',
 'world_centroid_alpha',
 'world_centroid_delta',
 'pixel_centroid_x',
 'pixel_centroid_y',
 'source_id',
 'detection_id',
 'auto_flux',
 'auto_flux_err',
 'auto_mag',
 'auto_mag_err',
 'auto_flags']

In [5]:
outside_r = (catalog['aperture_r_flags'] & stuff.SourceFlags.OUTSIDE)[:,:,0].astype(np.bool).all(axis=1)
in_all_r = np.logical_not(catalog['aperture_r_flags'] & stuff.SourceFlags.OUTSIDE)[:,:,0].astype(np.bool).all(axis=1)

In [6]:
outside_i = (catalog['aperture_i_flags'] & stuff.SourceFlags.OUTSIDE)[:,:,0].astype(np.bool).all(axis=1)
in_all_i = np.logical_not(catalog['aperture_i_flags'] & stuff.SourceFlags.OUTSIDE)[:,:,0].astype(np.bool).all(axis=1)

In [7]:
outside_all = np.logical_and(outside_i, outside_r)
inside_all = np.logical_and(in_all_r, in_all_i)

In [8]:
print("Inside all R frames:", sum(in_all_r), "/", len(catalog))
print("Outside all R frames:", sum(outside_r), "/", len(catalog))
print("Inside all I frames:", sum(in_all_i), "/", len(catalog))
print("Outside all I frames:", sum(outside_i), "/", len(catalog))
print("Outside all the frames:", sum(outside_all), "/", len(catalog))
print("Inside all the frames:", sum(inside_all), "/", len(catalog))

Inside all R frames: 284 / 753
Outside all R frames: 312 / 753
Inside all I frames: 186 / 753
Outside all I frames: 230 / 753
Outside all the frames: 0 / 753
Inside all the frames: 96 / 753


## Coadded image

In [9]:
coadd_img = fits.open(os.path.expanduser('~/Work/Data/sim08/coadd.fits'))[0].data

In [10]:
plt.figure(figsize=(8,8))
plt.imshow(coadd_img, norm=colors.SymLogNorm(10), cmap=plt.get_cmap('gray'))
plt.scatter(catalog[inside_all]['pixel_centroid_x'], catalog[inside_all]['pixel_centroid_y'], marker='*', c='yellow')

FigureCanvasNbAgg()